# JupyterHub Showcase: DevOps Intelligence

<img src="img/jhub-devops-intelligence.png" style="width: 95%; margin: 0;"></img>

<img src="https://avatars3.githubusercontent.com/u/1068245?s=40" alt="Avatar" style="float: left; padding-right:1rem; padding-bottom:.5rem;"></img>
📝 [Jürgen Hermann](https://twitter.com/jhermann_) · 📧 jh@web.de
<br />🗓️ March 2019

*DevOps Intelligence* turns data from software development and delivery processes into actionable insight, just like BI does for the business side. Jupyter is the ideal instrument for that, with its combination of powerful coding environments and a user interface facilitating experimentation with ultra-short feedback cycles.
<img src="img/jhub-logo-342px.png" style="float: right; padding-left: 1em;"></img>

A Jupyter-based setup supports risk analysis and decision making within development and operations processes – typical business intelligence / data science procedures can be applied to the ‘business of making and running software’. The idea is to create feedback loops, and facilitate human decision making by automatically providing reliable input in form of up-to-date facts. After all development is our business — so let's have KPIs for developing, releasing, and operating software.

## Typical Use-Cases

Here are some obvious application areas where data analysis can be helpful on the technical side.

* Migration processes of all kinds (current state, progress tracking, achievement of objectives)
* Inventory reporting for increased transparency and support of operational decisions
* Automate internal reporting processes to free up scarce assets and human expertise

## Platform Architecture

A simple [JupyterHub](https://jupyter.org/hub) setup can enable you to do analysis on your already available but under-used and hardly understood data, without any great investment of effort or capital. By adding a single JupyterHub host, you can use the built-in Python3 kernel to access existing internal data sources.

The following diagram shows what role JupyterHub can play in an existing environment.

> ![DevOps Intelligence Architecture](https://github.com/1and1/debianized-jupyterhub/raw/master/docs/_static/img/devops-intelligence.png)

To make such a deployment easy, the [1and1/debianized-jupyterhub](https://github.com/1and1/debianized-jupyterhub#jupyterhub-debian-packaging) project provides a JupyterHub service including a fully equipped Python3 kernel as a single Debian package – only Python3, NodeJS, and Chromium packages must be installed in addition to the `jupyterhub` one. If you raised an eyebrow on Chromium being in that list, it's used by JavaScript-based visualization frameworks to render PNG images.

Including a [NginX-powered SSL off-loader](https://github.com/1and1/debianized-jupyterhub#securing-your-jupyterhub-web-service-with-an-ssl-off-loader), the [complete setup](https://github.com/1and1/debianized-jupyterhub#how-to-set-up-a-simple-service-instance) can be done in under an hour.

## Use-Case: Migration Reporting

At the time of this writing (early 2019), a widespread challenge is migration from Oracle Java to other vendors, and also to start migration from Java 8 to newer versions (Java 11). If you do that at scale across many machines and teams, you definitely need some kind of governance, and constant feedback on the current status and the rate of progress.

What follows is an excerpt from a productive notebook, with anonymized data about [AdoptOpenJDK](https://adoptopenjdk.net/) deployments. That data was originally retrieved from a system called *“Patch Management Reporting”*, which collects information about installed packages for all hosts in the data center. We're in the yellow *“Data Sources”* box of the above figure here.

First off, we read the data and show the value sets of categorical columns, plus a sample.

In [0]:
import numpy as np
import pandas as pd

raw_data = pd.read_csv("../data/cmdb-aoj.csv", sep=',')
print('♯ of Records: {}\n'.format(len(raw_data)))

for name in raw_data.columns[1:]:
    if not name.startswith('Last '):
        print(name, '=', list(sorted(set(raw_data[name].fillna('')))))

print(); print(raw_data.head(3).transpose())

Next comes the usual data cleanup. The `Distribution` column is a bit diverse, and not everyone has Debian codenames and associated major versions memorized. The `map_distro` function fixes that.

In [0]:
def map_distro(name):
    """Helper to create canonical OS names."""
    return (name.split('.', 1)[0]
        .replace('Debian 7', 'wheezy')
        .replace('Debian 8', 'jessie')
        .replace('Debian 9', 'stretch')
        .replace('Debian 10', 'buster')
        .replace('squeeze', 'Squeeze [6]')
        .replace('wheezy', 'Wheezy [7]')
        .replace('jessie', 'Jessie [8]')
        .replace('stretch', 'Stretch [9]')
        .replace('buster', 'Buster [10]')
    )

Together with other cleanup steps, the mapper function is applied in a [dfply](https://towardsdatascience.com/dplyr-style-data-manipulation-with-pipes-in-python-380dcb137000) pipeline. The result can be controlled by showing a sample of data points with unique version numbers.

In [0]:
from dfply import *

cleaned = (raw_data
    >> mutate(Version=X['Installed version'].str.split('[()]', 1, expand=True)[0])
    >> mutate(Environment=X.Environment
              .fillna('--').str.replace('--', 'N/A').str.upper())
    >> mutate(Distribution=X.Distribution.apply(map_distro))
    >> drop(X.CMDB_Id, X['Last seen'], X['Last modified'], X['Installed version'])
)

print((cleaned >> distinct(X.Version)).transpose())

Time to present the cleaned up data, starting with a table of teams and their number of installed packages. In the production notebook, an API of the corporate identity management is used to enrich the table with contact data of the team leads. Having the organizational data available also makes it possible to filter or aggregate the data by business units.

In [0]:
counts = cleaned.groupby(['Team']).size()
print(counts.reset_index(name='Count'))

To create a heatmap of how diverse a team's version spectrum is, we calculate percentages of versions per team.

In [0]:
percentage = cleaned.groupby(['Team', 'Version']).size().reset_index(name='Count')
percentage = percentage.assign(Percent=
    percentage.apply(lambda x: 100.0 * x[-1] / counts[x[0]], axis=1))
print(percentage.head(3))

[HoloViews](http://holoviews.org/) makes creating the heatmap including a label overlay a breeze.

In [0]:
import holoviews as hv
hv.extension('bokeh')
publish = 1  # publishing or interactive mode?

heatmap = hv.HeatMap(percentage[['Version', 'Team', 'Percent', 'Count']]).opts(
    hv.opts.HeatMap(
        title='Version Distribution by Team',
        width=480, xrotation=25, 
        zlim=(0, 100), cmap='kbc_r', clipping_colors=dict(NaN='#ffffe0'),
        colorbar=True, tools=['hover'], toolbar=None if publish else 'right',
    )
).sort()

label_dimension = hv.Dimension('Percent', value_format=lambda x: '%.1f' % x)
labels = hv.Labels(heatmap, vdims=label_dimension).opts(
    hv.opts.Labels(
        text_color='Percent', 
        text_font_size='10pt',
        text_font_style='bold',
    )
)

chart = heatmap * labels

if publish:
    import time
    import phantomjs_bin
    from IPython.display import HTML, clear_output

    %env BOKEH_PHANTOMJS_PATH={phantomjs_bin.executable_path}
    chart_img = 'img/aoj-heatmap.png'
    hv.save(chart, chart_img)
    chart = HTML('<img src="{}?{}"></img>'.format(chart_img, time.time()))
    clear_output()

chart

From the heatmap, you can easily glance whether a team uses predominantly one version, and how recent the used versions are.

## Conclusion

Using a platform powered by [Project Jupyter](https://twitter.com/ProjectJupyter) and a big chunk of the scientific Python stack lets you easily mold your data into the shape you need, and then choose from a wide range of visualization options to bring your message across.